In [10]:
import chess
import chess.svg
import random
from IPython.display import display, SVG, clear_output
import json
from time import sleep
import subprocess

In [23]:
def random_move_game(board, num_moves=20):
    for _ in range(num_moves):
        legal_moves = list(board.legal_moves)
        if not legal_moves:
            break
        move = random.choice(legal_moves)
        board.push(move)
    return board

def numPieces( bitMap ):

   def sum_bits(num):
      return bin(num).count('1')

   return sum_bits(bitMap["rooks"]) + \
            sum_bits(bitMap["knights"]) + \
            sum_bits(bitMap["bishops"]) + \
            sum_bits(bitMap["queens"]) + \
            sum_bits(bitMap["kings"]) + \
            sum_bits(bitMap["pawns"])

def getPosFromNotation(notation: str, isWhite:bool):
    horizontalPos = ord(notation[1]) - ord('1')
    verticalPos = ord(notation[0]) - ord('a')
    return 1 << ((64*(not isWhite)) + (63 - (horizontalPos*8 + verticalPos)))


def isSameColorPiecePresent(board, pos, isWhiteMove:bool):
   for pieces in board:
      if board[pieces] & (1<<(64*(not isWhiteMove) + pos)):
         return True
   return False


def isOppositeColorPiecePresent(board, pos, isWhiteMove:bool):
    return isSameColorPiecePresent(board, pos, not isWhiteMove)

def isMovePossible(board, pos, isWhiteMove: bool):
   return not (pos >=64 or pos < 0 or isSameColorPiecePresent(board, pos, isWhiteMove))

def boardToBitMap( board ):

   pieceStructMap = {
      "r": "rooks",
      "n": "knights",
      "b": "bishops",
      "k": "kings",
      "q": "queens",
      "p": "pawns",
   }

   boardBitMap = {
      "rooks": 0,
      "knights": 0,
      "bishops": 0,
      "queens": 0,
      "kings": 0,
      "pawns": 0,
      "metadata": 0,
      "latest_move": 0
   }

   for index, piece in board.piece_map().items():
      isBlack = piece.symbol().islower()
      effIndex = 64*isBlack + ( 63 - index )
      boardBitMap[ pieceStructMap[ piece.symbol().lower() ] ] |= 1 << effIndex

   metadata = 0
   metadata |= board.fullmove_number << 16
   metadata |= board.halfmove_clock << 9
   metadata |= board.turn << 8
   en_passant_square = board.ep_square
   metadata |= bool( en_passant_square ) << 7
   if en_passant_square:
      column_number = chess.square_file(en_passant_square)
      metadata |= column_number << 4
   metadata |= board.has_kingside_castling_rights(0) << 3
   metadata |= board.has_queenside_castling_rights(0) << 2
   metadata |= board.has_kingside_castling_rights(1) << 1
   metadata |= board.has_queenside_castling_rights(1) << 0
   boardBitMap[ 'metadata' ] = metadata

   return boardBitMap

def bitMapFile( fileName, bitMap=None, isRead=True ):
   if isRead:
      with open(fileName, "r+") as f: 
         bitMap = json.load(f)
         return bitMap
   else:
      assert bitMap is not None, "No bitMap provided  to save!"
      with open(fileName, "w+") as f: 
         json.dump(bitMap, f, indent=2)

def bitMapToBoard( bitMap ):
   chessPieceMap = {
      "rooks": chess.ROOK,
      "knights": chess.KNIGHT,
      "bishops": chess.BISHOP,
      "kings": chess.KING,
      "queens": chess.QUEEN,
      "pawns": chess.PAWN
   }

   chessBoard = chess.Board()
   chessBoard.clear_board()

   for key, value in bitMap.items():
      if key not in [ 'metadata', 'latest_move' ]:
         for index in range(127, -1, -1):
            isPresent = value & 1 << index
            if isPresent:
               isWhite = index <= 63
               effIndex = 63 - ( index - ( not isWhite ) * 64 )
               chessPiece = chess.Piece( chessPieceMap[ key ], isWhite )
               chessBoard.set_piece_at( effIndex, chessPiece )
   
   castlingFen, metadata = '', bitMap[ 'metadata' ]
   chessBoard.fullmove_number = metadata >> 16
   chessBoard.halfmove_clock = metadata >> 9 & 127
   chessBoard.turn = bool( metadata & 1 << 8 )
   enPassantSquareExists = metadata & 1 << 7
   if enPassantSquareExists:
      rowNum = 5 if chessBoard.turn else 2
      chessBoard.ep_square = chess.square( metadata >> 4 & 7, rowNum )
   if metadata & 1 << 1: castlingFen += 'K'
   if metadata & 1 << 0: castlingFen += 'Q'
   if metadata & 1 << 3: castlingFen += 'k'
   if metadata & 1 << 2: castlingFen += 'q'
   chessBoard.set_castling_fen( castlingFen )

   return chessBoard

def display_board(board, size=800):
    svg_board = chess.svg.board(board, size=size)
    display(SVG(svg_board))

def extractMove( board, bitMap ):
   latestMove = bitMap[ 'latest_move' ]
   isKingSideCastle = ( latestMove >> 13 ) & 1
   if isKingSideCastle != 0:
      return '0-0'
   isQueenSideCastle = ( latestMove >> 12 ) & 1
   if isQueenSideCastle != 0:
      return "0-0-0"
   source = ( latestMove >> 6 ) & 63
   destination = latestMove & 63
   sourceX, sourceY = source%8, source//8
   destinationX, destinationY = destination%8, destination//8

   sourceLetters = chr(sourceX + ord('a')) + str(sourceY+1)
   destinationLetters = chr(destinationX + ord('a')) + str(destinationY+1)
   isOccupied = 'x' if board.piece_at(destination) is not None else ''

   # Check for pawn promotion
   isPawnPromotion = (latestMove >> 16) & 1
   promotionPiece = ''
   if isPawnPromotion:
      promotionType = (latestMove >> 14) & 3
      if promotionType == 0:
         promotionPiece = 'Q'
      elif promotionType == 1:
         promotionPiece = 'R'
      elif promotionType == 2:
         promotionPiece = 'B'
      elif promotionType == 3:
         promotionPiece = 'N'
      return f"{sourceLetters}"+f"{isOccupied}"+f"{destinationLetters}{promotionPiece}"

   return f"{sourceLetters}"+f"{isOccupied}"+f"{destinationLetters}"

In [24]:
bot_files = {
   "Latest Bot": "./target/release/mystic-bot",
   "Random Bot": "./robot/random-bot",
   "Piece Score": "./robot/piece-score-bot",
   "Position Score": "./robot/position-score-bot",
   "Sorted Boards": "./robot/sorted-eval-bot"
}

In [25]:
def run_game( bot1, bot2, targetJsonFile, display=True, startFen=None, bot1White=None, maxMoves=float('inf') ):

   assert bot1 in bot_files and bot2 in bot_files, "Bots not Found!!"

   chessBoard = chess.Board() if not startFen else chessBoard( startFen )
   bitMapFile( targetJsonFile, boardToBitMap( chessBoard ), isRead=False )

   if bot1White is None:
      bot1White = random.choice( [True, False] ) 

   whiteBot, blackBot = ( bot1, bot2 ) if bot1White else ( bot2, bot1 )
   numMoves, result = 0, None
   while not chessBoard.can_claim_draw() and not chessBoard.is_checkmate() and numMoves < maxMoves:

      if display:
         clear_output()
         print( "BLACK BOT:", blackBot )
         display_board( chessBoard )
         print( "WHITE BOT:", whiteBot )
         print("-"*50)
         print("WHITE BEST EVALUATION")
         print(result.stdout if result else '')

      result = subprocess.run(
            [bot_files[ whiteBot ], targetJsonFile],
            check=True,
            capture_output=display,
            text=True,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.STDOUT
      )
      move = extractMove( chessBoard, bitMapFile( targetJsonFile ) )
      chessBoard.push_san( move )

      if chessBoard.can_claim_draw() or chessBoard.is_checkmate():
         break

      if display:
         clear_output()
         print( "BLACK BOT:", blackBot )
         display_board( chessBoard )
         print( "WHITE BOT:", whiteBot )
         print("-"*50)
         print("BLACK BEST EVALUATION")
         print(result.stdout)

      result = subprocess.run(
         [bot_files[ blackBot ], targetJsonFile],
         check=True,
         capture_output=display,
         text=True
      )
      move = extractMove( chessBoard, bitMapFile( targetJsonFile ) )
      chessBoard.push_san( move )

   if display:
      clear_output()
      print( "BLACK BOT:", blackBot )
      display_board( chessBoard )
      print( "WHITE BOT:", whiteBot )
      print("-"*50)
      print("BLACK BEST EVALUATION")
      print(result.stdout)

   result = chessBoard.result()
   if result == "1-0":
    if display: print("WHITE WINS!", whiteBot)
    return whiteBot, 1
   elif result == "0-1":
      if display: print("BLACK WINS!", blackBot)
      return blackBot, -1
   else:
      if display:
         print( "DRAW!" )
      return "Draw", 0

In [26]:
run_game( "Sorted Boards", "Random Bot", "./sample/position.json" )

('Sorted Boards', 1)